In [61]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
!pip install transformers
!pip install datasets
!pip install evaluate

In [20]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import os

In [21]:
dataset = load_dataset("json",data_files={"train":"/content/treino.jsonl", "test":"/content/teste.jsonl"})

In [22]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 500
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 100
    })
})


In [25]:
checkpoint= "bert-base-uncased"

# conceito: é necessario pegar um texto, aplicar o embedding nele para virar codigo/numero
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#classificação
mapDict= {
    "venda": 0,
    "suporte": 1
}

def transform_labels(label):
  label = label['completion']
  result = []
  for l in label:
    result.append(mapDict[l])

  return {"label": result}

def tokenize_function(example):
  return tokenizer(example['prompt'], padding=True, truncation=True)

In [26]:
# roda a tokenizacao no dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

#roda a transformação das labels considerando o mapDict
tokenized_datasets = tokenized_datasets.map(transform_labels, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [36]:
# definicao dos argumentos para treino
from transformers import TrainingArguments

output_dir = "./bert-categoria-venda-suporte-test"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir="./logs.log",
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=200,
    save_total_limit=2,
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)


In [41]:
from transformers import Trainer,AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3)

# desabilita comportamento default do huggingface pra utilizar o wandb para analise de modelos e pesos
os.environ["WANDB_DISABLE"] = "true"
os.environ["WANDB_MODE"] = "offline"

⚙️  Running in WANDB offline mode


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [55]:
trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    data_collator = data_collator,
    processing_class = tokenizer,
    compute_metrics = compute_metrics
)

In [56]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
200,0.000100,0.000043,1.000000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=315, training_loss=0.004590347912975602, metrics={'train_runtime': 1084.0626, 'train_samples_per_second': 2.306, 'train_steps_per_second': 0.291, 'total_flos': 33403070610000.0, 'train_loss': 0.004590347912975602, 'epoch': 5.0})

In [57]:
trainer.evaluate()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 4.341030944488011e-05,
 'eval_accuracy': 1.0,
 'eval_runtime': 36.0353,
 'eval_samples_per_second': 2.775,
 'eval_steps_per_second': 0.361,
 'epoch': 5.0}

In [58]:
trainer.save_model()

In [62]:
trainer.push_to_hub("ecn3/model-venda-suporte")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...te-test/model.safetensors:   0%|          | 14.2kB /  438MB            

  ...te-test/training_args.bin:  11%|#         |   630B / 5.78kB            

CommitInfo(commit_url='https://huggingface.co/ecn3/bert-categoria-venda-suporte-test/commit/11b2f129ddd5ad4148e52cd27fffe21874891ac6', commit_message='ecn3/model-venda-suporte', commit_description='', oid='11b2f129ddd5ad4148e52cd27fffe21874891ac6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ecn3/bert-categoria-venda-suporte-test', endpoint='https://huggingface.co', repo_type='model', repo_id='ecn3/bert-categoria-venda-suporte-test'), pr_revision=None, pr_num=None)

In [64]:
from transformers import pipeline
pipe = pipeline("text-classification", model= "ecn3/bert-categoria-venda-suporte-test")

config.json:   0%|          | 0.00/835 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [66]:
pipe("Poderia me informar sobre as promoções de colchões?")

[{'label': 'LABEL_0', 'score': 0.999955415725708}]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')